In [1]:
import re
import json
from bs4 import BeautifulSoup
import unicodedata
import time

In [2]:
import requests
import json
import html
import re
from bs4 import BeautifulSoup

def post_request(url, headers):
    response = requests.post(url, headers=headers)
    return response.json()


def post_request_key(url, headers, data):
    try:
        data = requests.post(url, headers=headers, data=json.dumps(data))
        data.raise_for_status()  # Raise an error for bad status codes
        return data.json()
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None


In [3]:
# Sử dụng hàm để thực hiện yêu cầu POST
def get_api():
    url1 = 'https://apids.thuvienphapluat.vn/auth/get-token?key=pvaG4gRG9lIiwiaWF0IjoxNTE2MjM5MDIyfQS563xADXH'
    headers1 = {
        'Cookie': 'Culture=vi; Culture=vi'
    }

    key = post_request(url1, headers1)

    url = 'https://apids.thuvienphapluat.vn/data/get-hoidapphapluat'
    headers = {
        'Authorization': 'Bearer ' + key['Data']['AccessToken'],
        'Content-Type': 'application/json',
        'Cookie': 'Culture=vi'
    }
    return url, headers

def send_data_to_api(data):
    url, headers = get_api()
    response = requests.post(url, headers=headers, json=data)
    return response.json()

In [4]:
def remove_text_between(text, start_word, end_word):
    # Create a regex pattern to match text between start_word and end_word
    pattern = re.compile(f'{start_word}.*?{end_word}', re.DOTALL)
    # Substitute the matched text with an empty string
    result = pattern.sub('', text)
    return result

In [5]:
def extract_data(data_input):
    data_real = []
    answer = BeautifulSoup(data_input['content'], 'html.parser')
    source = {
        "question": data_input['title'],
        "answer": remove_text_between(answer.text, "Bạn", "thân mến,"),
        "links": [],
        "ccpls": []
    }
    data_real.append(source)
    patterns = [
        re.compile(r"\s*\n{2}\s*Về\s*vấn\s*đề\s*này,"),  # Khớp với ít nhất 2 ký tự xuống dòng trước và giữa các từ
        re.compile(r"Về vấn đề này,\s*THƯ VIỆN PHÁP LUẬT\s*giải đáp như sau:"),
        re.compile(r"\*\*\*\*", re.MULTILINE),
        re.compile(r"THƯ VIỆN PHÁP LUẬT"),
        re.compile(r"^Xem thêm.*", re.MULTILINE),
        re.compile(r"^>> Xem thêm.*", re.MULTILINE),
        re.compile(r"^Xem chi tiết tại.*", re.MULTILINE),
        re.compile(r"^>> Tham khảo.*", re.MULTILINE),
        re.compile(r"^Tham khảo.*", re.MULTILINE),
        re.compile(r"^Chọn lĩnh vực để xem văn bản liên quan.*", re.MULTILINE),
        re.compile(r"^Trân trọng.*", re.MULTILINE),
        re.compile(r"^Bài được viết theo.*", re.MULTILINE),
        re.compile(r"^Xem toàn bộ.*", re.MULTILINE),
        re.compile(r"^Xem bài viết.*", re.MULTILINE)
    ]

    # Vòng lặp qua từng cặp câu hỏi và câu trả lời
    for qa_pair in data_real:
        # Lấy câu trả lời từ cặp câu hỏi và câu trả lời
        answer = qa_pair["answer"]
        # Thay thế từ "tại đây" thành rỗng trong câu trả lời
        cleaned_answer = re.sub(r'.*>\s*tại đây\s*<.*', '', answer, flags=re.IGNORECASE | re.MULTILINE)
        # Cập nhật câu trả lời đã được thay thế trong cặp câu hỏi và câu trả lời
        qa_pair["answer"] = cleaned_answer

    for item in data_real:
        answer_text = item["answer"]
        # Khởi tạo start_index với giá trị -1 (không tìm thấy)
        start_index = -1
        # Kiểm tra từng mẫu regex và cập nhật start_index nếu tìm thấy
        for pattern in patterns:
            match = pattern.search(answer_text)
            if match:
                index = match.start()
                # Nếu start_index chưa được cập nhật hoặc tìm thấy index nhỏ hơn (gần đầu chuỗi hơn)
                if start_index == -1 or index < start_index:
                    start_index = index

        # Nếu tìm thấy bất kỳ mẫu nào, xóa từ đoạn đó đến hết câu trả lời
        if start_index != -1:
            cleaned_answer = answer_text[:start_index].strip()
            item["answer"] = cleaned_answer
    filtered_questions_and_answers = [qa for qa in data_real if 'answer' in qa and qa['answer']]
    filtered_questions_and_answers1 = [qa for qa in filtered_questions_and_answers if 'question' in qa and qa['question']]
    return filtered_questions_and_answers1

In [6]:
data_api = {
    "page": 1, #14039
    "num": 1,
    "field":0,
    "expireccpl": 2
}
url, headers = get_api()
response = post_request_key(url, headers, data_api)
file_note = []
data = []
if response:
    specific_page = response.get('total')
# # Số trang cần lấy
# specific_page =  298847 #46146 pldn ko có tbvbm
# print(now)
num = 100
total_page = specific_page // num + ( 1 if specific_page % num > 0 else 0)
for num_page in range(total_page, 1, -1):
    data_api = {
        "page": num_page, #14039
        "num": num,
        "field":0,
        "expireccpl": 2
    }
    url1 = 'https://apids.thuvienphapluat.vn/auth/get-token?key=pvaG4gRG9lIiwiaWF0IjoxNTE2MjM5MDIyfQS563xADXH'
    headers1 = {
        'Cookie': 'Culture=vi; Culture=vi'
    }

    key = post_request(url1, headers1)
    url_post = "https://apids.thuvienphapluat.vn/crud/log-extracted-news-data"

    headers_post = {
    'Authorization': 'Bearer ' + key['Data']['AccessToken'],
    'Content-Type': 'application/json',
    'Cookie': 'Culture=vi'
    }

    url, headers = get_api()
    response = post_request_key(url, headers, data_api)
    # print(response.get('data'))
    if response:
        data_input = response.get('data')
        for temp in data_input:
#             # print(temp["content"])
            extracted_data = extract_data(temp)
#             # print(extract_data(temp["content"]))
            if int(temp['obj_id']) < 373545 and extracted_data != [] and not re.search(r'\b(?:dự thảo|đề xuất|sắp tới|dự kiến|đáp án|tra cứu điểm thi)\b', temp["title"].lower(), flags=re.IGNORECASE):
                dictionary = {
                    "objid": int(temp['obj_id']),
                    "source": str(temp['obj_code']),
                    "data": json.dumps(extracted_data, ensure_ascii=False),
                    "type": 2
                }
                dictionary = json.dumps(dictionary, ensure_ascii=False)
                data.append(dictionary)
                # print(dictionary)
                response = requests.request("POST", url_post, headers=headers_post, data=dictionary)
                print(response.text)
            elif int(temp['obj_id']) < 373545 and extracted_data != [] and re.search(r'\b(?:dự thảo|đề xuất|sắp tới|dự kiến|đáp án|tra cứu điểm thi)\b', temp["title"].lower(), flags=re.IGNORECASE):
                dictionary = {
                    "objid": int(temp['obj_id']),
                    "source": str(temp['obj_code']),
                    "data": json.dumps(extracted_data, ensure_ascii=False),
                    "type": 3
                }
                dictionary = json.dumps(dictionary, ensure_ascii=False)
                data.append(dictionary)
                # print(dictionary)
                response = requests.request("POST", url_post, headers=headers_post, data=dictionary)
                print(response.text)
            else:
                file_note.append((temp['obj_id'], temp['id']))
                # time.sleep(1)
# now = specific_page

{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages":"success"}
{"messages"

In [ ]:
with open('data_miss.txt', 'w') as file:
    for item in file_note:
        file.write(f"{item}\n")

In [ ]:
with open('data_ok.txt', 'w') as file:
    for item in data:
        file.write(f"{item}\n")